## Fase 1: Extracción de datos de películas

In [1]:
## Importación de librerías

import pandas as pd
import numpy as np
import requests
import mysql.connector
from mysql.connector import errorcode
pd.set_option('display.max_columns', None) 
import warnings
warnings.filterwarnings("ignore")

In [2]:
## Obtención datos: 100 películas

url = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

datos = requests.get(url)
print(datos.status_code)
print(datos.content)

datos_json = datos.json()

200
b'[\n    {\n        "id": 1,\n        "titulo": "The Godfather",\n        "a\xc3\xb1o": 1972,\n        "duracion": 175,\n        "genero": "Crimen",\n        "adultos": false,\n        "subtitulos": [\n            "es",\n            "en"\n        ]\n    },\n    {\n        "id": 2,\n        "titulo": "The Godfather Part II",\n        "a\xc3\xb1o": 1974,\n        "duracion": 202,\n        "genero": "Crimen",\n        "adultos": false,\n        "subtitulos": [\n            "es",\n            "en"\n        ]\n    },\n    {\n        "id": 3,\n        "titulo": "Pulp Fiction",\n        "a\xc3\xb1o": 1994,\n        "duracion": 154,\n        "genero": "Crimen",\n        "adultos": true,\n        "subtitulos": [\n            "es",\n            "en"\n        ]\n    },\n    {\n        "id": 4,\n        "titulo": "Forrest Gump",\n        "a\xc3\xb1o": 1994,\n        "duracion": 142,\n        "genero": "Drama",\n        "adultos": false,\n        "subtitulos": [\n            "es",\n            

In [3]:
print (type(datos_json))
print (len(datos_json)) ## para saber cuántas hay ya que se pide extraer 100 películas

<class 'list'>
100


In [4]:
df = pd.DataFrame(datos_json)
df

,id,titulo,año,duracion,genero,adultos,subtitulos
0,1,The Godfather,1972,175,Crimen,False,"[es, en]"
1,2,The Godfather Part II,1974,202,Crimen,False,"[es, en]"
2,3,Pulp Fiction,1994,154,Crimen,True,"[es, en]"
3,4,Forrest Gump,1994,142,Drama,False,"[es, en, fr]"
4,5,The Dark Knight,2008,152,Acción,False,"[es, en]"
...,...,...,...,...,...,...,...
95,96,La vita è bella,1997,116,Drama,False,"[es, en, it]"
96,97,Requiem for a Dream,2000,102,Drama,True,"[es, en]"
97,98,Memento,2000,113,Thriller,True,"[es, en]"
98,99,Eternal Sunshine of the Spotless Mind,2004,108,Drama,False,"[es, en]"


In [5]:
las_100_pelis = datos_json[:100]

In [6]:
df_100 = pd.DataFrame(las_100_pelis)

## Fase 2: Creación de la Base de Datos

In [7]:
cnx = mysql.connector.connect(user='root', password='1234',
                              host='localhost',
                              database ='examen_apis',auth_plugin='mysql_native_password')

mycursor = cnx.cursor()
print("Conexión OK:", cnx.is_connected())

Conexión OK: True


In [8]:
df_pelis = df_100[["titulo", "año", "duracion", "genero", "adultos"]]
df_pelis.head()

,titulo,año,duracion,genero,adultos
0,The Godfather,1972,175,Crimen,False
1,The Godfather Part II,1974,202,Crimen,False
2,Pulp Fiction,1994,154,Crimen,True
3,Forrest Gump,1994,142,Drama,False
4,The Dark Knight,2008,152,Acción,False


In [10]:
mycursor.execute("USE examen_apis")

query = '''CREATE TABLE IF NOT EXISTS peliculas (
    id INT PRIMARY KEY AUTO_INCREMENT,
    titulo VARCHAR(255) NOT NULL,
    año INT,
    duracion INT,
    genero VARCHAR(50),
    adultos BOOLEAN
);
'''

mycursor.execute(query)
cnx.commit()  ## para que se guarden los datos

## Fase 3: Inserción de los Datos en la Base de Datos

In [ ]:
df_to_insert = df_100[["titulo", "año", "duracion", "genero", "adultos"]].copy() # para no usar columnas que no se van a inssertar

df_clean = df_to_insert.replace({np.nan: None})   # Reemplazar NaN con None para que se inserten como NULL en la BD

df_clean["adultos"] = df_clean["adultos"].map({True: 1, False: 0, None: None}) # para convertir el booleano a 1 = Adultos /0 o no existe

datos = df_clean[["titulo", "año", "duracion", "genero", "adultos"]].values.tolist()   # Convertir DataFrame directamente a lista de tuplas

query_insert = """
INSERT INTO peliculas (titulo, año, duracion, genero, adultos) 
VALUES (%s, %s, %s, %s, %s)
"""
mycursor.executemany(query_insert, datos)  # insertar muchos
cnx.commit()

print(f"{mycursor.rowcount} registros insertados")

100 registros insertados


##  Fase 4: Obtener información a partir de los datos.
Una vez que tenemos toda la información, vamos a responder las siguientes preguntas utilizando consultas en SQL

In [ ]:
## Una de las consultas pide datos de los subtítulos que no estaban en la tabla inicial. 
# Hay que hacer una modificación de la talbla.

# Función para comprobar si una columna existe
def columna_existe(nombre_columna):
    mycursor.execute("SHOW COLUMNS FROM peliculas;")
    columnas = [col[0] for col in mycursor.fetchall()]
    return nombre_columna in columnas

print("Existe 'subtitulos'? ->", columna_existe("subtitulos"))
print("Existe 'subtitulos_es'? ->", columna_existe("subtitulos_es"))

In [ ]:
# Añadir columna subtitulos (TEXT)
try:
    mycursor.execute("ALTER TABLE peliculas ADD COLUMN subtitulos TEXT;")
    print("Columna 'subtitulos' creada.")
except Exception as e:
    print("Error creando columna subtitulos:", e)

# Añadir columna subtitulos_es (BOOLEAN)
try:
    mycursor.execute("ALTER TABLE peliculas ADD COLUMN subtitulos_es BOOLEAN DEFAULT 0;")
    print("Columna 'subtitulos_es' creada.")
except Exception as e:
    print("Error creando columna subtitulos_es:", e)

cnx.commit()

# Verificamos la estructura
mycursor.execute("SHOW COLUMNS FROM peliculas;")
for fila in mycursor.fetchall():
    print(fila)

In [12]:
# DataFrame base (asegúrate de que es el original con 'subtitulos') ---
df_subs = df_100[["titulo", "año", "subtitulos"]].copy()

# Normalizar nulos
df_subs["subtitulos"] = df_subs["subtitulos"].where(df_subs["subtitulos"].notna(), None)

# Función para transformar la columna subtitulos en JSON y detectar 'es'
def procesa_subs(x):
    if x is None:
        return None, None
    # Si viene como string que contiene una lista (p. ej. "['es','en']") intentar parsear
    if isinstance(x, str):
        try:
            parsed = json.loads(x)
            if isinstance(parsed, (list, tuple)):
                x = parsed
        except Exception:
            # no es JSON: intentar limpieza básica (quita corchetes y split por coma)
            s = x.strip()
            if s.startswith("[") and s.endswith("]"):
                s = s[1:-1]
            elems = [e.strip().strip("'\"") for e in s.split(",") if e.strip()]
            if elems:
                x = elems
    # A estas alturas x debería ser lista/tuple o algo no válido
    if isinstance(x, (list, tuple, set)):
        elems_norm = [str(e).lower() for e in x]
        tiene_es = 1 if "es" in elems_norm or "spa" in elems_norm else 0
        # Serializar a JSON para guardar como TEXT (mantener acentos)
        try:
            json_str = json.dumps(list(x), ensure_ascii=False)
        except Exception:
            # fallback: unir con comas
            json_str = ",".join(map(str, x))
        return json_str, tiene_es
    # Si no es lista, devolver None
    return None, None

# Aplicar la función y crear dos columnas nuevas
df_subs[["subtitulos_json", "subtitulos_es"]] = df_subs["subtitulos"].apply(
    lambda z: pd.Series(procesa_subs(z))
)

# Comprobaciones rápidas
print("Total filas:", len(df_subs))
print("Filas con subtitulos no nulos:", df_subs["subtitulos_json"].notna().sum())
print("Detectadas con 'es':", df_subs["subtitulos_es"].sum())
print("\nEjemplo (primeras 5 filas):")
display(df_subs.head())

# Preparar UPDATE por fila 
update_query = """
UPDATE peliculas
SET subtitulos = %s,
    subtitulos_es = %s
WHERE titulo = %s AND `año` = %s
"""

# Ejecutar actualización fila por fila y detectar no-coincidencias
misses = []   # filas que no actualizaron (posible mismtach)
updated_count = 0

for idx, row in df_subs.iterrows():
    vals = (row["subtitulos_json"], 
            int(row["subtitulos_es"]) if row["subtitulos_es"] is not None else None,
            row["titulo"],
            int(row["año"]) if not pd.isna(row["año"]) else None)
    # Ejecutar
    mycursor.execute(update_query, vals)
    if mycursor.rowcount == 0:
        misses.append((idx, row["titulo"], row["año"]))
    else:
        updated_count += mycursor.rowcount

# Confirmar cambios
cnx.commit()
print(f"\nTotal filas intentadas: {len(df_subs)}")
print(f"Total filas actualizadas (sumadas rowcount): {updated_count}")
print(f"Filas sin coincidencia (no actualizadas): {len(misses)}")

if misses:
    print("\nEjemplos de títulos no actualizados (posible mismatch de título/año):")
    for i, t, y in misses[:10]:
        print(f"  - idx {i} | {t} | {y}")

# Verificación final: cuántas tienen subtitulos_es = 1
mycursor.execute("SELECT COUNT(*) FROM peliculas WHERE subtitulos_es = 1;")
print("\nPelículas con subtítulos en español (subtitulos_es = 1):", mycursor.fetchone()[0])

# (Opcional) para ver algunas filas con subtítulos guardados
mycursor.execute("SELECT titulo, `año`, subtitulos, subtitulos_es FROM peliculas LIMIT 10;")
for r in mycursor.fetchall():
    print(r)


Total filas: 100
Filas con subtitulos no nulos: 100
Detectadas con 'es': 100

Ejemplo (primeras 5 filas):


,titulo,año,subtitulos,subtitulos_json,subtitulos_es
0,The Godfather,1972,"[es, en]","es,en",1
1,The Godfather Part II,1974,"[es, en]","es,en",1
2,Pulp Fiction,1994,"[es, en]","es,en",1
3,Forrest Gump,1994,"[es, en, fr]","es,en,fr",1
4,The Dark Knight,2008,"[es, en]","es,en",1



Total filas intentadas: 100
Total filas actualizadas (sumadas rowcount): 100
Filas sin coincidencia (no actualizadas): 0

Películas con subtítulos en español (subtitulos_es = 1): 100
('The Godfather', 1972, 'es,en', 1)
('The Godfather Part II', 1974, 'es,en', 1)
('Pulp Fiction', 1994, 'es,en', 1)
('Forrest Gump', 1994, 'es,en,fr', 1)
('The Dark Knight', 2008, 'es,en', 1)
('Fight Club', 1999, 'es,en', 1)
('Inception', 2010, 'es,en,de', 1)
('The Matrix', 1999, 'es,en', 1)
('The Shawshank Redemption', 1994, 'es,en', 1)
('Interstellar', 2014, 'es,en', 1)
